In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from modules.nyc_shapefiles import read_nyc_shapefiles

In [2]:
precinct_dict, tract_dict, merged_census_info = read_nyc_shapefiles()
print('Mean number of tracts per precinct: {0}'.format(
    np.mean([len(v['intersections']) for k, v in precinct_dict.items()])
))
print('Mean number of precincts per tract: {0}'.format(
    np.mean([len(v['intersections']) for k, v in tract_dict.items()])
))

Mean number of tracts per precinct: 40.103896103896105
Mean number of precincts per tract: 1.4263279445727484


In [3]:
print('---Keys---')
for k, v in tract_dict['1407'].items():
    print(k, type(v))
print()

tract_dict['1407']

---Keys---
shape_orig <class 'shapely.geometry.polygon.Polygon'>
shape_gps <class 'shapely.geometry.polygon.Polygon'>
intersections <class 'dict'>
properties <class 'collections.OrderedDict'>
census_info <class 'pandas.core.series.Series'>
total_intersection_area <class 'float'>



{'census_info': GEO.id2                                                  36005036100
 GEO.display-label           Census Tract 361, Bronx County, New York
 Population                                                      6099
 Median_Household_Income                                        18799
 Percent_Bachelors_Degree                                         7.8
 Name: 1400000US36005036100, dtype: object,
 'intersections': {'30': {'intersection_area_absolute': 1760194.0239685476,
   'intersection_area_percent_of_tract': 1.0}},
 'properties': OrderedDict([('CTLabel', '361'),
              ('BoroCode', '2'),
              ('BoroName', 'Bronx'),
              ('CT2010', '036100'),
              ('BoroCT2010', '2036100'),
              ('CDEligibil', 'E'),
              ('NTACode', 'BX17'),
              ('NTAName', 'East Tremont'),
              ('PUMA', '3705'),
              ('Shape_Leng', 5701.06188867),
              ('Shape_Area', 1760194.02397)]),
 'shape_gps': <shapely.geometry.pol

In [4]:
print('---Keys---')
for k in precinct_dict['50']:
    print(k)
print()

precinct_dict['50']

---Keys---
shape_orig
shape_gps
intersections
properties
total_population



{'intersections': {'100': {'population_assigned_to_precinct': 1181.8216337416325,
   'population_percent_of_precinct': 0.012841349882442173},
  '104': {'population_assigned_to_precinct': 1927.2504157944732,
   'population_percent_of_precinct': 0.020940974673094751},
  '106': {'population_assigned_to_precinct': 204.34002173250809,
   'population_percent_of_precinct': 0.0022203026574707536},
  '1117': {'population_assigned_to_precinct': 2139.6336508152008,
   'population_percent_of_precinct': 0.02324867268115334},
  '1118': {'population_assigned_to_precinct': 2449.1770533113008,
   'population_percent_of_precinct': 0.026612086433082542},
  '1216': {'population_assigned_to_precinct': 442.23527710202677,
   'population_percent_of_precinct': 0.0048052072846615388},
  '1232': {'population_assigned_to_precinct': 0.00070366883070973338,
   'population_percent_of_precinct': 7.645872607615594e-09},
  '1240': {'population_assigned_to_precinct': 2375.0,
   'population_percent_of_precinct': 0.02580

In [5]:
print(np.sum([d['total_population'] for d in precinct_dict.values()]))
print(np.sum(merged_census_info['Population']))
print(np.sum([d['census_info']['Population'] for d in tract_dict.values()]))

8426743.0
8426743
8426743


In [6]:
precinct_dict['50']['properties']

OrderedDict([('Precinct', 79),
             ('Shape_Leng', 28256.6021125),
             ('Shape_Area', 44975546.9316)])

In [11]:
precinct_dict['50']['shape_gps'].geometry()

AttributeError: 'Polygon' object has no attribute 'geometry'

In [12]:
import shapely

In [14]:
def create_feature(key, val):
    pass

def cfc(shape_dict):
    output_dict = {
        'type': 'FeatureCollection',
        'features': []
    }

{'coordinates': (((-73.93856854848711, 40.69884813554975),
   (-73.93848766732174, 40.69880516733192),
   (-73.93817718618821, 40.69864022329228),
   (-73.93815708957668, 40.69851472448597),
   (-73.9381357896041, 40.69838170757393),
   (-73.93810517170144, 40.69819049720775),
   (-73.938088711252, 40.698087702300356),
   (-73.93807151221566, 40.698002610777095),
   (-73.9379544821177, 40.69742359221613),
   (-73.93794236786951, 40.69736914868103),
   (-73.93792316380201, 40.69728284063561),
   (-73.93779110792102, 40.696689342233164),
   (-73.93763027449141, 40.69595086258838),
   (-73.93867614288104, 40.69583093541051),
   (-73.93951197570928, 40.69573529040102),
   (-73.93970150313743, 40.69571360091015),
   (-73.94055524905511, 40.69561589745617),
   (-73.94041185850952, 40.69488089726512),
   (-73.94026548739568, 40.69414839746769),
   (-73.94011786576063, 40.69341620400914),
   (-73.9400882372574, 40.693267924474334),
   (-73.94006857803832, 40.69316954431201),
   (-73.9399714421